# EDA (Exploratory Data Analysis) with graphics

## Imports & settings

In [2]:

import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

plt.rcParams["figure.dpi"] = 120
plt.rcParams["savefig.dpi"] = 300

FIG_DIR = "report/figures"
os.makedirs(FIG_DIR, exist_ok=True)

TRAIN_PATH = "data/CW1_train.csv"
TEST_PATH  = "data/CW1_test.csv"


## Load data

## Basic schema checks

## Quick descriptive stats

## Identify numeric columns

## Duplicate rows check

## Target (outcome) distribution

## Categorical distributions + check levels in train vs test

## Outcome by category (boxplots)

## Numeric feature distributions (hist grid)

## Correlation with target (numeric only)

## Correlation heatmap (top correlated numeric features)

## Scatter plots vs outcome for top correlated numeric features

## Outlier check (IQR) for key numeric variables

## Train vs Test distribution drift check (numeric)

## Train vs Test: quick overlay hist for a few important features

## Notes you can paste into your report (auto-generated text summary)